# Cluster 2

In [31]:
import pandas as pd # pandas
import numpy as np # numpy
import matplotlib.pyplot as plt  # matplotlib
import seaborn as sns  # seaborn
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [32]:
df = pd.read_csv('../data_updated_clustered_kmeans.csv')

In [ ]:
df['cluster'].value_counts()

### Criação do PCA

In [ ]:
features = ["faixa-etaria_encoded","valor-pago-sinistro_standardized", "descricao_servico_sinistro_encoded", "doenca_relacionadas_encoded", "sexo_encoded"]
X = df[features]

kmeans = KMeans(n_clusters=6, random_state=0)
kmeans.fit(X)
df['clus'] = kmeans.predict(X)

# Aplicar PCA para reduzir os dados a 2 componentes principais
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Adicionar os componentes principais ao DataFrame para visualização
df['PCA1.2'] = X_pca[:, 0]
df['PCA2.2'] = X_pca[:, 1]

# Plotar os resultados em um gráfico de dispersão
plt.figure(figsize=(20, 6))
sns.scatterplot(x='PCA1.2', y='PCA2.2', hue='clus', data=df, palette='tab10')

# # Plotar os centróides dos clusters em cima do gráfico PCA
# centroids_pca = pca.transform(kmeans.cluster_centers_)
# plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], s=300, c='red', marker='X', label='Centroides')

plt.title('Visualização dos Clusters com PCA e KMeans')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.show()

### Introdução

&emsp;Durante a elaboração do gráfico de dispersão pelo modelo K-means, foram selecionadas seis *features*: ```faixa-etaria_encoded```,```valor-pago-sinistro_standardized```, ```descricao_servico_sinistro_encoded```, ```doenca_relacionadas_encoded```, ```sexo_encoded```, ```tipo_servico_encoded```. Examinando os nomes citados, é possível perceber que a grande parte contém "_encoded" no final, mostrando que advêm de uma coluna categórica não numérica, a qual teve seus elementos codificados em números para poder ser utilizada em análises.

&emsp;Embora as *features* utilizadas possam ser listadas, é difícil dizer o que cada eixo representa. Isso ocorre devido ao uso da técnica PCA (Análise de Componenetes Principais), reponsável por reduzir a dimensionalidade dos dados. Ela foi extremamente útil nesse caso, uma vez que cada colunas definida anteriormente seria uma dimensão, porém não é possível elaborar um gráfico 6D. Aplicando o PCA, foi possível reduzir a duas dimensões, sendo elas elaboradas a partir de uma combinação linear de todas as variáveis originais.

&emsp;Para identificar o que cada eixo indica, foi elaborada uma análise de influência, ou seja, o quanto cada *feature* foi levada em consideração ao desenvolver os eixos dos gráficos:

In [ ]:
loadings = pca.components_
loadings_X = pd.DataFrame(loadings.T, columns=['PC1', 'PC2'], index=X.columns)
print(loadings_X)

&emsp;Ao investigar os resultados, se torna possível inferir que o eixo x é praticamente definido pelo serviço utilizado, e o eixo y pela doença ralacionada a esses serviços, ambas as *features* com aproximadamente 99% de influência. Essas informações facilitam a interpretação do gráfico, pois dispensam uma comparação profunda para a interpretação dos dados.

&emsp;Ao analisar o eixo y, entretanto, será necessária uma atenção a mais. Considerando o resultado negativo apresentado pela coluna ```doenca_relacionadas_encoded``` para "PCA2", é importante ter em mente que as duas variáveis são inversamente relacionadas, ou seja, conforme uma aumenta, a outra diminui.

&emsp;Nesse contexto, abaixo será relizada uma análise de cada cluster encontrado pelo modelo, identificando as características que os compõem:

### Análise do Cluster 2

&emsp;Agora, será analisado o *cluster* 2, o qual pode ser encontrado no início da região positiva do eixo x. Nele, são encontrados os elementos representados pelos valores de 29 a 332 no eixo x, e de -27 a 201 no eixo y, como pode ser visto abaixo:

In [ ]:
c2 = df[df['cluster'] == 2]

# plt.figure(figsize=(10, 6))
sns.scatterplot(x=c2['PCA1'], y=c2['PCA2'])

plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('Scatterplot do Cluster 2')
plt.show()

In [ ]:
# Filtrando apenas as linhas identificadas com "2" na coluna "cluster"
cluster2 = df[df['cluster'] == 2]

print(f'Valor máximo no eixo x: {cluster2['PCA1'].nlargest(1).values[0]}')
print(f'Valor mínimo no eixo x: {cluster2['PCA1'].nsmallest(1).values[0]}', '\n')
print(f'Valor máximo no eixo y: {cluster2['PCA2'].nlargest(1).values[0]}')
print(f'Valor mínimo no eixo y: {cluster2['PCA2'].nsmallest(1).values[0]}')

&emsp;Esses valores englobam cerca de 301 serviços e 53 doenças, as quais serão explorados posteriormente.

In [ ]:
print(f'Número de serviços no eixo x: {cluster2['Descricao Servico Sinistro'].nunique()}')
print(f'Número de doenças no eixo y: {cluster2['Doença relacionada'].nunique()}')

&emsp;O gráfico a seguir mostra o dez serviços mais frequentes dentro do *cluster* 2:

In [ ]:
servicos_top10 = c2['Descricao Servico Sinistro'].value_counts().nlargest(10)

colors = ['brown', 'blue', 'orange', 'green', 'purple', 'red', 'pink', 'gray', 'yellow', 'cyan']  # 10 cores

plt.figure(figsize=(10, 6))
bars = sns.barplot(x=servicos_top10.index, y=servicos_top10.values, palette=colors, hue=servicos_top10.index)

# Remove os rótulos do eixo X
bars.set_xticklabels([''] * len(servicos_top10))

# Coloca a legenda fora do gráfico (bbox_to_anchor define a posição fora do gráfico)
handles = [plt.Rectangle((0, 0), 1, 1, color=colors[i]) for i in range(len(servicos_top10))]
plt.legend(handles, servicos_top10.index, title='Serviços', loc='lower left', bbox_to_anchor=(1, 0.5), fontsize='small')

# Adicionando rótulos e título
plt.xlabel('Serviços')  # Remove o rótulo do eixo x
plt.ylabel('Frenquência')
plt.title('Dez Serviços mais Frequentes no Cluster 2')

# plt.tight_layout()  # Ajusta para evitar cortes
plt.show()


&emsp;Como é possível notar, "material" é o serviço mais frenquente, alcançando pouco mais de 1600 ocorrências. Já para as doenças relacionadas, o gráfico pode ser visto abaixo:

In [ ]:
doencas_top10 = c2['Doença relacionada'].value_counts().nlargest(10)

colors = ['brown', 'blue', 'orange', 'green', 'purple', 'red', 'pink', 'gray', 'yellow', 'cyan']  # 10 cores

plt.figure(figsize=(10, 6))
bars = sns.barplot(x=doencas_top10.index, y=doencas_top10.values, palette=colors, hue=doencas_top10.index)

# Remove os rótulos do eixo X
bars.set_xticklabels([''] * len(doencas_top10))

# Coloca a legenda fora do gráfico (bbox_to_anchor define a posição fora do gráfico)
handles = [plt.Rectangle((0, 0), 1, 1, color=colors[i]) for i in range(len(doencas_top10))]
plt.legend(handles, doencas_top10.index, title='Serviços', loc='upper right', fontsize='small')

# Adicionando rótulos e título
plt.xlabel('Doenças')  # Remove o rótulo do eixo x
plt.ylabel('Frenquência')
plt.title('Dez Doenças mais Frequentes no Cluster 2')

# plt.tight_layout()  # Ajusta para evitar cortes
plt.show()


- Doenças Ósseas

In [ ]:
copia = c2.copy()
copia['Doença relacionada'] = copia['Doença relacionada'].replace(['Osteoartrite', 'Osteoartrite (Artrose)', 'Doenças osteomioarticulares'], 'Doenças Ósseas')
copia[copia['Doença relacionada'] == 'Doenças Ósseas']['Descricao Servico Sinistro'].value_counts()

- Doenças Cardiovasculares

In [ ]:
copia['Doença relacionada'] = copia['Doença relacionada'].replace(['Hipertensão arterial', 'Insuficiência cardíaca', 'Diabetes'], 'Doenças Cardiovasculares')
copia[copia['Doença relacionada'] == 'Doenças Cardiovasculares']['Descricao Servico Sinistro'].value_counts()

&emsp;De forma discrepante dos valores apresentados na coluna de descrição serviço sinistro, doenças ósseas representam a maior parte dos dados, com doenças cardiovasculares, em menor número, em seguida. Isso se dá, pois os serviços mais utilizados do cluster não possuem doenças claramente relacionadas. Nesse contexto, é perceptível uma forte tendência do *cluster* para os casos de uma certa frequência de doenças ósseas, sendo uma informação útil para a análise do grupo. 

### Insights

##### Entendimento das colunas 

As doenças cardiovasculares e ósseas foram algo que se destacou por serem as mais frequentes nesse conjunto de dados, por isso será feita uma análise em cima das colunas que categorizam o tipo de usuário do plano de saúde, que são a de sexo e a de faixa-etária.

In [ ]:
copia['Doença relacionada'].value_counts().head(5)

Será feita agora uma análise sobre as proporções de cada sexo e faixa etária para depois, entender se as doenças cumprem as proporções de população esperada dentro do _cluster_ ou se existe alguma tendência de tipo de usuário para cada doença.

- Sexo:

In [ ]:
proporcao_sexo_cluster2 = c2['Sexo Sinistro'].value_counts(normalize=True)
proporcao_sexo_geral = df['Sexo Sinistro'].value_counts(normalize=True)

print('Proporção de Homens e Mulheres no Cluster 2:')
print(proporcao_sexo_cluster2)
print('Proporção de Homens e Mulheres no Geral:')
print(proporcao_sexo_geral)

Como afirmado anteriormente, para o _cluster 2_ sexo não foi um fator significativo de divisão, algo que se comprova em função da proporção de homens X mulheres se manter nesse conjunto de dados. Uma proporção de aproximadamente 60/40 entre mulheres e homens. 

- Faixa-Etária:

In [ ]:
proporcao_faixa_etaria_cluster2 = c2['Faixa-Etária Nova Sinistro'].value_counts(normalize=True)
proporcao_faixa_etaria_geral = df['Faixa-Etária Nova Sinistro'].value_counts(normalize=True)
proporcao_faiza_etaria = proporcao_faixa_etaria_cluster2 / proporcao_faixa_etaria_geral

print('Proporção de Faixa Etária Geral X Cluster:')
print(proporcao_faiza_etaria)

Quanto mais próximo os valores de 1 quer dizer que a proporção dentro do _cluster_ se segue. A maioria das faixas segue o esperado, apesar das idades de 19 a 38 anos serem ligeiramente menores nesse conjunto de dados.

##### Doenças Cardiovasculares

In [ ]:
df_cardiovascular = copia[copia['Doença relacionada'] == 'Doenças Cardiovasculares']
sexo_cardiovascular = df_cardiovascular['Sexo Sinistro'].value_counts(normalize=True)

print('Proporção de Homens e Mulheres nas doenças cardiovasculares:')
print(sexo_cardiovascular)

Pode-se ver que a proporção não se cumpre para as doenças cardiovasculares, se cumpre, na realidade, uma proporção de 52/48, logo, mesmo estando contido no conjunto de dados um valor superior de mulheres, ao observar isoladamente essas doenças o número de homens é proporcionalmente maior do que o esperado.

In [ ]:
ordem = ['0 a 18 anos', '19 a 23 anos', '24 a 28 anos', '29 a 33 anos', '34 a 38 anos', '39 a 43 anos', '44 a 48 anos', '49 a 53 anos', '54 a 58 anos', '59 anos ou mais']
sns.catplot(x='Faixa-Etária Nova Sinistro', kind='count', data=df_cardiovascular, order=ordem, hue='Sexo Sinistro', width=0.5, height=7, aspect=1.75)
plt.ylabel('Quantidade')
plt.title('Faixa Etária - Doenças Cardiovasculares')
plt.show()

Ao visualizar o gráfico, infere-se que faixas-etárias mais elevadas tem uma maior tendência a possuir problemas cardiovasculares, principalmente a faixa de '59 anos ou mais'. Além disso, deve-se atentar também ao valor elevado de homens, mesmo sendo minoria no conjunto de dados do _cluster 2_, quando se observa as doenças cardiovasculares os homens se apresentam em pé de igualdade com as mulheres, sendo superiores em algumas faixas, algo que deve ser observado atentamente. Seguindo nesse tópico, os homens ainda superam as mulheres em diversas faixas, fazendo com que eles sejam maioria em alguns momentos, algo que deve passar por uma análise de entendimento, uma vez que se esperava que os homens tivessem uma tendência menor a ir ao médico, algo que não se cumpre nessa análise.

##### Doenças Ósseas

In [ ]:
df_osseo = copia[copia['Doença relacionada'] == 'Doenças Ósseas']
sexo_osseo = df_osseo['Sexo Sinistro'].value_counts(normalize=True)

print('Proporção de Homens e Mulheres nas Doenças Ósseas:')
print(sexo_osseo)

Da mesma forma das informações vistas nas doenças cardiovasculares, a proporção de sexo do _cluster_ não se cumpre aqui. Com uma proporção de 52/48, aproximadamente, as doenças ósseas tem um número de homens acima do esperado para esse conjunto dados.

In [ ]:
ordem = ['0 a 18 anos', '19 a 23 anos', '24 a 28 anos', '29 a 33 anos', '34 a 38 anos', '39 a 43 anos', '44 a 48 anos', '49 a 53 anos', '54 a 58 anos', '59 anos ou mais']
sns.catplot(x='Faixa-Etária Nova Sinistro', kind='count', data=df_osseo, order=ordem, hue='Sexo Sinistro', width=0.5, height=7, aspect=1.75)
plt.ylabel('Quantidade')
plt.title('Faixa Etária - Doenças Ósseas')
plt.show()

A primeira observação notável é que, na faixa etária de 49 a 53 anos, há uma predominância expressiva de casos em mulheres, superando bastante o número de ocorrências em homens. Isso sugere que, nesta faixa etária, as mulheres estão sendo mais afetadas por doenças ósseas do que os homens, uma vez que estão realizando mais exames relacionados à saúde ósseas.

Além disso, pode se ver um claro aumento de quantidade de usos a partir dos 34 anos, ou seja, uma concentração maior nas faixas-etárias superiores, algo natural, uma vez que pessoas mais velhos tem uma tendência maior a ter problemas ósseas.

Outra informação a se pontuar é de que apesar de estar em menor número na base e nos _cluster_ os homens, a partir de 34 anos (com exceção da faixa-etária de 49-53 anos), são superiores em todas as categorias. Mostrando que existe uma tendência maior aos homens mais velhos de fazerem mais atendimentos médicos relacionados a doenças ósseas.

Por fim, é importante adicionar que a faixa-etária de 0-18 anos apresenta valores pequenos de problemas ósseos e quase totalmente composta por homens. Uma hipótese relacionada a isso é a de que a possível prática de esportes de maneira regular, feita majoritariamente por homens nessa idade, pode implicar em uma tendência ao uso do sinistro para problemas ósseos nessa idade.


### Conclusão

&emsp;Em suma, o *cluster* 2 apresenta um forte direcionamento para casos de doenças osteomioarticulares e cardiovasculares, além do uso de materiais hospitalares e compra de medicamentos, sendo os dois valores mais frequentes em suas respectivas categorias. Entretanto, eles não possuem grande correlação entre si, uma vez que os valores de descrição de sinsitro não representam nenhuma das doenças tidas como mais recorrentes. Apesar disso, existe essa recorrência de dois conjuntos de doenças específicas que são as ósseas e cardiovasculares.

&emsp;Na faixa etária de 49 a 53 anos, as mulheres apresentam uma predominância expressiva de atendimentos relacionados a doenças ósseas, enquanto, a partir dos 34 anos, exceto entre 49 e 53 anos, os homens superam as mulheres, sugerindo uma maior procura por cuidados ósseos entre homens mais velhos. O aumento dos atendimentos é mais evidente em faixas etárias superiores, como esperado, devido à maior tendência de problemas ósseos com a idade. Na faixa de 0 a 18 anos, há poucos casos, quase todos entre homens, possivelmente ligados à prática esportiva regular.

&emsp;Já ao analisar as doenças cardiovasculares, pode-se perceber que a maioria dos valores dos casos estão contidos na faixa-etária de mais de 59 anos, o que comprova o maior uso desses serviços por usuários mais velhos do plano.